In [1]:
from pyspark.sql import SparkSession
import os


In [2]:
spark = SparkSession.builder \
        .master("local[12]") \
        .appName("app") \
        .config("spark.driver.memory", "45g") \
        .config("spark.executor.memory", "45g") \
        .config("spark.jars", "postgresql-42.3.3.jar") \
        .getOrCreate()
#spark.sparkContext.setLogLevel("DEBUG")

23/10/19 15:35:34 WARN Utils: Your hostname, server-rennweg resolves to a loopback address: 127.0.1.1; using 192.168.1.196 instead (on interface enp5s0f1)
23/10/19 15:35:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/19 15:35:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/19 15:35:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
spark.sparkContext.setLogLevel("WARN") # ALL, DEBUG, WARN,
#spark.sparkContext.setLogLevel("ALL") # ALL, DEBUG, WARN,

In [3]:
username = os.environ.get('USERNAME', 'dsb')
password = os.environ.get('PASSWORD', 'dsb')
dbname = os.environ.get('DBNAME', 'dsb')
dbhost = os.environ.get('DBHOST', 'localhost')

df_tables = spark.read.format("jdbc") \
    .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "information_schema.tables") \
    .option("user", username) \
    .option("password", password) \
    .load()

for idx, row in df_tables.toPandas().iterrows():
        if row.table_schema == 'public':
            table_name = row.table_name
            df = spark.read.format("jdbc") \
                .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
                .option("driver", "org.postgresql.Driver") \
                .option("dbtable", table_name) \
                .option("user", username) \
                .option("password", password) \
                .load()
    
            print(table_name)
            print(df.show())
            df.createOrReplaceTempView(table_name)

dbgen_version
+----------+--------------+-------------------+---------------+
|dv_version|dv_create_date|     dv_create_time|dv_cmdline_args|
+----------+--------------+-------------------+---------------+
|    2.11.0|    2023-10-03|1970-01-01 16:38:35|     -scale 10 |
+----------+--------------+-------------------+---------------+

None
customer_address
+-------------+----------------+----------------+----------------+---------------+---------------+------------+----------------+--------+----------+-------------+-------------+--------------------+
|ca_address_sk|   ca_address_id|ca_street_number|  ca_street_name| ca_street_type|ca_suite_number|     ca_city|       ca_county|ca_state|    ca_zip|   ca_country|ca_gmt_offset|    ca_location_type|
+-------------+----------------+----------------+----------------+---------------+---------------+------------+----------------+--------+----------+-------------+-------------+--------------------+
|            1|AAAAAAAABAAAAAAA|      71        |

23/10/19 15:35:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+---------+-----------------+--------------------+--------------------+----------------+------------+---------------------+--------------------+
|cd_demo_sk|cd_gender|cd_marital_status| cd_education_status|cd_purchase_estimate|cd_credit_rating|cd_dep_count|cd_dep_employed_count|cd_dep_college_count|
+----------+---------+-----------------+--------------------+--------------------+----------------+------------+---------------------+--------------------+
|         1|        M|                M|Primary             |                 500|      Good      |           0|                    0|                   0|
|         2|        F|                M|Primary             |                 500|      Good      |           0|                    0|                   0|
|         3|        M|                S|Primary             |                 500|      Good      |           0|                    0|                   0|
|         4|        F|                S|Primary             |   

+---------------+---------------+-------+-------+----------+-----------+
|sm_ship_mode_sk|sm_ship_mode_id|sm_type|sm_code|sm_carrier|sm_contract|
+---------------+---------------+-------+-------+----------+-----------+
+---------------+---------------+-------+-------+----------+-----------+

None
time_dim
+---------+----------------+------+------+--------+--------+-------+--------------------+--------------------+-----------+
|t_time_sk|       t_time_id|t_time|t_hour|t_minute|t_second|t_am_pm|             t_shift|         t_sub_shift|t_meal_time|
+---------+----------------+------+------+--------+--------+-------+--------------------+--------------------+-----------+
|        0|AAAAAAAABAAAAAAA|     0|     0|       0|       0|     AM|third               |night               |       NULL|
|        1|AAAAAAAACAAAAAAA|     1|     0|       0|       1|     AM|third               |night               |       NULL|
|        2|AAAAAAAADAAAAAAA|     2|     0|       0|       2|     AM|third     

call_center
+-----------------+-----------------+-----------------+---------------+-----------------+---------------+-------+--------+------------+--------+--------+----------+---------+------------+-----------+-----------------+-----------+----------------+----------+---------------+----------------+--------------+--------------+---------------+-------+---------+--------+------+----------+-------------+-----------------+
|cc_call_center_sk|cc_call_center_id|cc_rec_start_date|cc_rec_end_date|cc_closed_date_sk|cc_open_date_sk|cc_name|cc_class|cc_employees|cc_sq_ft|cc_hours|cc_manager|cc_mkt_id|cc_mkt_class|cc_mkt_desc|cc_market_manager|cc_division|cc_division_name|cc_company|cc_company_name|cc_street_number|cc_street_name|cc_street_type|cc_suite_number|cc_city|cc_county|cc_state|cc_zip|cc_country|cc_gmt_offset|cc_tax_percentage|
+-----------------+-----------------+-----------------+---------------+-----------------+---------------+-------+--------+------------+--------+--------+-------

...

+-------------------+-------------------+----------+-----------------------+--------------------+--------------------+-------------------+------------------------+---------------------+---------------------+--------------------+-----------------+------------------+---------------+---------------+------------+---------------+------------------+----------------+-------------+---------------------+------+-------------------+----------------+------------------+---------------+-----------+
|cr_returned_date_sk|cr_returned_time_sk|cr_item_sk|cr_refunded_customer_sk|cr_refunded_cdemo_sk|cr_refunded_hdemo_sk|cr_refunded_addr_sk|cr_returning_customer_sk|cr_returning_cdemo_sk|cr_returning_hdemo_sk|cr_returning_addr_sk|cr_call_center_sk|cr_catalog_page_sk|cr_ship_mode_sk|cr_warehouse_sk|cr_reason_sk|cr_order_number|cr_return_quantity|cr_return_amount|cr_return_tax|cr_return_amt_inc_tax|cr_fee|cr_return_ship_cost|cr_refunded_cash|cr_reversed_charge|cr_store_credit|cr_net_loss|
+-------------------

+-------------------+-------------------+----------+-----------------------+--------------------+--------------------+-------------------+------------------------+---------------------+---------------------+--------------------+--------------+------------+---------------+------------------+-------------+-------------+---------------------+------+-------------------+----------------+------------------+-----------------+-----------+
|wr_returned_date_sk|wr_returned_time_sk|wr_item_sk|wr_refunded_customer_sk|wr_refunded_cdemo_sk|wr_refunded_hdemo_sk|wr_refunded_addr_sk|wr_returning_customer_sk|wr_returning_cdemo_sk|wr_returning_hdemo_sk|wr_returning_addr_sk|wr_web_page_sk|wr_reason_sk|wr_order_number|wr_return_quantity|wr_return_amt|wr_return_tax|wr_return_amt_inc_tax|wr_fee|wr_return_ship_cost|wr_refunded_cash|wr_reversed_charge|wr_account_credit|wr_net_loss|
+-------------------+-------------------+----------+-----------------------+--------------------+--------------------+------------

+---------------+---------------+---------------+----------+-------------------+----------------+----------------+---------------+-------------------+----------------+----------------+---------------+--------------+--------------+---------------+---------------+-----------+---------------+-----------+-----------------+-------------+--------------+-------------------+------------------+---------------------+-----------------+----------+-------------+----------------+-----------+-------------------+--------------------+------------------------+-------------+
|ws_sold_date_sk|ws_sold_time_sk|ws_ship_date_sk|ws_item_sk|ws_bill_customer_sk|ws_bill_cdemo_sk|ws_bill_hdemo_sk|ws_bill_addr_sk|ws_ship_customer_sk|ws_ship_cdemo_sk|ws_ship_hdemo_sk|ws_ship_addr_sk|ws_web_page_sk|ws_web_site_sk|ws_ship_mode_sk|ws_warehouse_sk|ws_promo_sk|ws_order_number|ws_quantity|ws_wholesale_cost|ws_list_price|ws_sales_price|ws_ext_discount_amt|ws_ext_sales_price|ws_ext_wholesale_cost|ws_ext_list_price|ws_ext_ta

+---------------+---------------+---------------+-------------------+----------------+----------------+---------------+-------------------+----------------+----------------+---------------+-----------------+------------------+---------------+---------------+----------+-----------+---------------+-----------+-----------------+-------------+--------------+-------------------+------------------+---------------------+-----------------+----------+-------------+----------------+-----------+-------------------+--------------------+------------------------+-------------+
|cs_sold_date_sk|cs_sold_time_sk|cs_ship_date_sk|cs_bill_customer_sk|cs_bill_cdemo_sk|cs_bill_hdemo_sk|cs_bill_addr_sk|cs_ship_customer_sk|cs_ship_cdemo_sk|cs_ship_hdemo_sk|cs_ship_addr_sk|cs_call_center_sk|cs_catalog_page_sk|cs_ship_mode_sk|cs_warehouse_sk|cs_item_sk|cs_promo_sk|cs_order_number|cs_quantity|cs_wholesale_cost|cs_list_price|cs_sales_price|cs_ext_discount_amt|cs_ext_sales_price|cs_ext_wholesale_cost|cs_ext_list_p

+---------------+---------------+----------+--------------+-----------+-----------+----------+-----------+-----------+----------------+-----------+-----------------+-------------+--------------+-------------------+------------------+---------------------+-----------------+----------+-------------+-----------+-------------------+-------------+
|ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|ss_net_paid_inc_tax|ss_net_profit|
+---------------+---------------+----------+--------------+-----------+-----------+----------+-----------+-----------+----------------+-----------+-----------------+-------------+--------------+-------------------+------------------+---------------------+-----------------+----------+-------------+-----------+

In [ ]:
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()
spark.sql("SET spark.sql.yannakakis.enabled = true").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = true").show()
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.cbo.enabled = true").show()

In [ ]:
spark.sql("ANALYZE TABLE part COMPUTE STATISTICS;").show()

In [ ]:
df = spark.sql("""select sum(ss_quantity) / sum(ss_quantity) from store_sales,store where s_store_sk = ss_store_sk""")
df.show()

df.explain(True)

In [ ]:
df = spark.sql("""select avg(i_current_price) from item, inventory where i_item_sk = inv_item_sk""")
df = spark.sql("""select avg(ss_quantity) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select avg(ss_list_price) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(i_current_price)) from item, inventory where i_item_sk = inv_item_sk""")
#df = spark.sql("""select sum(if (isnull(ss_quantity), 0, ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")

df.show()

df.explain(True)

In [ ]:
def run_query(file):
    with open(file, 'r') as f:
        query = '\n'.join(filter(lambda line: not line.startswith('limit') and not line.startswith('-'), f.readlines()))
        
        print("running query: \n" + query)
        return spark.sql(query)

In [ ]:
df = spark.sql("""
SELECT ps_partkey, count(*) from part, partsupp
WHERE p_partkey = ps_partkey
GROUP BY ps_partkey
""")

df.show()

df.explain(True)

In [ ]:
df = spark.sql("SELECT COUNT(p_size) / COUNT(DISTINCT p_size) FROM part")
df.show()
df = spark.sql("SELECT COUNT(p_retailprice) / COUNT(DISTINCT p_retailprice) FROM part")
df.show()

In [ ]:
df_t1 = spark.createDataFrame([(1,1), (2,1), (2,2), (3,2), (3,3), (4,3), (4,3), (5,2), (5,1), (6,4)], schema=("a","b"))
df_t1.createOrReplaceTempView("t1")
df_t2 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("c","d"))
df_t2.createOrReplaceTempView("t2")
df_t3 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("e","f"))
df_t3.createOrReplaceTempView("t3")

query = "select median(a) from t1, t2 where b = c"
#query = "select percentile(a, 0.5, b) from t1, t2 where b = c"
#query = "select median(a) from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"
#query = "select count(*) from t1, t2 where b = c"
#query = "select *a from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

df = spark.sql(query)
df.show()

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

df = spark.sql(query)
df.show()

In [ ]:
spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "6")
#spark.conf.set("spark.executor.instances", "6")
spark.conf.set("spark.sql.shuffle.partitions", "6")

In [ ]:
import pandas as pd
import time

def benchmark(query):
    df0 = run_query(query)
    df0.show()
    
    spark.sql("SET spark.sql.yannakakis.enabled = true").show()

    start_time = time.time()

    df1 = run_query(query)
    df1.show()
    #df1.explain(True)

    end_time = time.time()
    yannakakis_time = end_time - start_time

    spark.sql("SET spark.sql.yannakakis.enabled = false").show()

    start_time = time.time()

    df2 = run_query(query)
    df2.show()
    #df2.explain(True)

    end_time = time.time()
    ref_time = end_time - start_time
    
    #return [query, ref_time, yannakakis_time]
    return [query, ref_time, yannakakis_time]


queries = ['tpch-kit/dbgen/queries/postgres/2.sql',
           'tpch-kit/dbgen/queries/postgres/11.sql', 
           'tpch-kit/dbgen/queries/postgres/11-hint.sql',
           'median-1.sql',
           'median-2.sql', 
           'median-3.sql', 
           'median-4.sql', 
           'median-5.sql',
            'median-1-hint.sql',
           'median-2-hint.sql', 
           'median-3-hint.sql', 
           'median-4-hint.sql', 
           'median-5-hint.sql']

results = [benchmark(q) for q in queries]

df = pd.DataFrame(results, columns = ['query', 'ref_time', 'yannakakis_time'])

print(df)

df.to_csv("results.csv")
    

#print(f'row count: {df1.count()} vs. {df2.count()}' )
    #print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')

In [ ]:
#spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "1")
#spark.conf.set("spark.executor.instances", "1")
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [ ]:
## Compare result
import time
#query = 'tpch-kit/dbgen/queries/postgres/2.sql'
#query = 'tpch-kit/dbgen/queries/postgres/13.sql'
#query = 'count-3.sql'
#query = 'tpch-kit/dbgen/queries/postgres/11.sql'
#query = '11-simple.sql'
query = 'median-2-hint.sql'
#query = 'tpch-kit/dbgen/queries/postgres/7.sql'
#query = '13-simple.sql'
query = 'dsb-queries/agg_queries/query013.sql'

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

start_time = time.time()

df1 = run_query(query)
df1.show()
df1.explain(mode="extended")

end_time = time.time()
yannakakis_time = end_time - start_time

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

start_time = time.time()

df2 = run_query(query)
df2.show()
df2.explain(mode="extended")

end_time = time.time()
ref_time = end_time - start_time

#print(f'row count: {df1.count()} vs. {df2.count()}' )
print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')